**Задание 1**  
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
  - оценка 2 и меньше - низкий рейтинг  
  - оценка 4 и меньше - средний рейтинг  
  - оценка 4.5 и 5 - высокий рейтинг  
Результат классификации запишите в столбец class

In [44]:
import pandas as pd
from IPython.display import display, HTML

def range_rating(val):
    if val <= 2:
        return "низкий"
    elif val <= 4:
        return "средний"
    elif val == 4.5 or val == 5:
        return "высокий"
    else:
        return "прочий"
    
def add_class():
    try:
        ratings = pd.read_csv('ml-latest-small/ratings.csv')
    except:
        return

    ratings['class'] = ratings['rating'].apply(range_rating)
    return ratings


ratings = add_class()
if ratings is None:
    print("Что-то пошло не так")
else:
    display(HTML(ratings.head(10).to_html()))

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний
1,1,1029,3.0,1260759179,средний
2,1,1061,3.0,1260759182,средний
3,1,1129,2.0,1260759185,низкий
4,1,1172,4.0,1260759205,средний
5,1,1263,2.0,1260759151,низкий
6,1,1287,2.0,1260759187,низкий
7,1,1293,2.0,1260759148,низкий
8,1,1339,3.5,1260759125,средний
9,1,1343,2.0,1260759131,низкий


**Задание 2**  
Используем файл keywords.csv.  
Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т.е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.  
Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
```python
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
```
Результат классификации запишите в отдельный столбец region.

In [45]:
def get_region(val, fmap):
    for reg, cities in fmap.items():
        for city in cities:
            if city in val:
                return reg
    return "undefined"


def add_region(fmap):
    try:
        keywords = pd.read_csv('keywords.csv')
    except:
        return
    
    keywords['region'] = keywords['keyword'].apply(get_region, fmap=fmap)
    return keywords


geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
    
keywords = add_region(geo_data)
if keywords is None:
    print("Что-то пошло не так")
else:
    # выведем только первые 10 из тех, которые смогли определить
    display(HTML(keywords[ keywords['region'] != "undefined" ].head(10).to_html()))

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад
1259,москва,153050,Центр
1346,санкт петербург,144232,Северо-Запад
1396,гисметео москва,140279,Центр
1482,госуслуги москва,133061,Центр


**Задание 3 (бонусное)**  
Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.
При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:
1. В переменную years запишите список из всех годов с 1950 по 2010.
2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
  - для каждой строки пройдите по всем годам списка years
  - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
  - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'
4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [46]:
def production_year(title, years):
    for year in years:
        if "(" + str(year) + ")" in title:
            return year
    return 1900

def worse_or_not():
    try:
        ratings = pd.read_csv('ml-latest-small/ratings.csv')
        movies = pd.read_csv('ml-latest-small/movies.csv')
    except:
        return

    # добавим инфу о рейтинге
    movies_plus = pd.merge(movies, ratings[['movieId', 'rating']], on='movieId')
    
    # далее по алгоритму
    years = [x for x in range(1950, 2011)]
    movies_plus['year'] = movies_plus['title'].apply(production_year, years=years)
    mn = (movies_plus
        .groupby('year')['rating']
        .mean()
        .reset_index()
        .sort_values('rating', ascending=False)
    )
    
    return mn


won = worse_or_not()
if won is None:
    print("Что-то пошло не так")
else:
    display(HTML(won.to_html()))

,year,rating
8,1957,4.014241
23,1972,4.011136
3,1952,4.000000
5,1954,3.994220
2,1951,3.983539
25,1974,3.978704
13,1962,3.952446
1,1950,3.915254
28,1977,3.905786
15,1964,3.841492
